# NN Baselines

In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
from google.colab import drive
drive.mount('gdrive')
import string

Mounted at gdrive


In [3]:
!nvidia-smi

Sat Nov 27 07:03:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
cd /content/gdrive/MyDrive/College/Semester5/NLP/project

/content/gdrive/MyDrive/College/Semester5/NLP/project


In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# df_train = pd.read_csv("train.csv")
# df_train['author'] = df_train['author'].replace(np.nan, "<NO_AUTHOR>")
# df_train['title'] =  df_train['title'].replace(np.nan, "<NO_TITLE>")
# df_train['text'] =   df_train['text'].replace(np.nan, "<NO_TEXT>")
# for i, j in df_train.iterrows():
#     df_train.at[i, 'text'] = j['author']+" | "+j['title'] +" | "+ j['text']
# def punctuation_removal(text):
#     all_list = [char for char in text if char not in string.punctuation]
#     clean_str = ''.join(all_list)
#     return clean_str
# df_train['text'] = df_train['text'].apply(punctuation_removal)
# stop = stopwords.words('english')
# df_train['text'] = df_train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# df_train = df_train.sample(frac=1)

In [ ]:
# del df_train['author']
# del df_train['title']
# del df_train['id']

In [ ]:
# df = df_train.sample(frac=1)

In [ ]:
# from sklearn.model_selection import train_test_split
# train_x, valid_x, train_y, valid_y = train_test_split(df['text'], df['label'], test_size = 0.2)

In [6]:
# import pickle
# file_to_store = open("split.pkl", "wb")
# pickle.dump((train_x, valid_x, train_y, valid_y), file_to_store)

In [7]:
from sklearn.model_selection import train_test_split
file = open("split.pkl", 'rb')
train_x, valid_x, train_y, valid_y = pickle.load(file)

In [8]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [9]:
from tensorflow.keras import optimizers

In [10]:
# encoder = preprocessing.LabelEncoder()
# train_y = encoder.fit_transform(train_y)
# valid_y = encoder.fit_transform(valid_y)

In [11]:
trainDF = pd.DataFrame()
trainDF['text'] = np.concatenate([train_x, valid_x])
trainDF['labels'] = np.concatenate([train_y, valid_y])

In [12]:
embeddings_index = {}
for i, line in enumerate(open('/content/gdrive/MyDrive/Amazon/productner/data/glove.6B.100d.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

embedding_matrix = numpy.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):

    classifier.fit(feature_vector_train, label, epochs=3)

    predictions = classifier.predict(feature_vector_valid)
    pred =[]
    for i in predictions:
        pred.append([np.round(i[0])])
    predictions = pred
    return metrics.accuracy_score(predictions, valid_y), metrics.precision_score(predictions, valid_y),metrics.recall_score(predictions, valid_y),metrics.f1_score(predictions, valid_y)

In [26]:
def create_cnn():

    input_layer = layers.Input((70, ))

    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=True)(input_layer)
    # embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    output_layer2 = layers.Dense(1, activation="sigmoid")(pooling_layer)

    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)

print ("accuracy:-", accuracy[0])
print ("precision:-", accuracy[1])
print ("recall:-", accuracy[2])
print ("f1 score:-", accuracy[3])


Epoch 1/3
520/520 [==============================] - 16s 30ms/step - loss: 0.2718
Epoch 2/3
520/520 [==============================] - 16s 30ms/step - loss: 0.0805
Epoch 3/3
520/520 [==============================] - 16s 30ms/step - loss: 0.0219
accuracy:- 0.9504807692307692
precision:- 0.968019093078759
recall:- 0.9358560221504384
f1 score:- 0.9516658845612389


In [31]:
def create_rcnn():

    input_layer = layers.Input((70, ))

    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    rnn_layer = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(embedding_layer)
    # rnn_layer = layers.GRU(50, return_sequences=True)(embedding_layer)


    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("accuracy:-", accuracy[0])
print ("precision:-", accuracy[1])
print ("recall:-", accuracy[2])
print ("f1 score:-", accuracy[3])


Epoch 1/3
520/520 [==============================] - 4s 6ms/step - loss: 0.4652
Epoch 2/3
520/520 [==============================] - 3s 6ms/step - loss: 0.2590
Epoch 3/3
520/520 [==============================] - 3s 6ms/step - loss: 0.2117
accuracy:- 0.926923076923077
precision:- 0.9665871121718377
recall:- 0.896414342629482
f1 score:- 0.9301791456132292


In [33]:
def create_bidirectional_rnn():

    input_layer = layers.Input((70, ))

    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("accuracy:-", accuracy[0])
print ("precision:-", accuracy[1])
print ("recall:-", accuracy[2])
print ("f1 score:-", accuracy[3])


Epoch 1/3
520/520 [==============================] - 33s 56ms/step - loss: 0.3582
Epoch 2/3
520/520 [==============================] - 29s 55ms/step - loss: 0.1417
Epoch 3/3
520/520 [==============================] - 29s 55ms/step - loss: 0.0672
accuracy:- 0.9533653846153847
precision:- 0.9646778042959427
recall:- 0.9439514245679589
f1 score:- 0.9542020774315393


In [35]:
def create_rnn_gru():

    input_layer = layers.Input((70, ))

    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    lstm_layer = layers.GRU(100)(embedding_layer)

    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    # output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("accuracy:-", accuracy[0])
print ("precision:-", accuracy[1])
print ("recall:-", accuracy[2])
print ("f1 score:-", accuracy[3])


Epoch 1/3
520/520 [==============================] - 12s 19ms/step - loss: 0.4013
Epoch 2/3
520/520 [==============================] - 10s 19ms/step - loss: 0.2245
Epoch 3/3
520/520 [==============================] - 10s 19ms/step - loss: 0.1798
accuracy:- 0.9348557692307692
precision:- 0.9274463007159904
recall:- 0.9422890397672163
f1 score:- 0.93480875631465


In [ ]:
def create_rnn_lstm():
    input_layer = layers.Input((70, ))

    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)


    lstm_layer = layers.LSTM(100)(embedding_layer)

    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("accuracy:-", accuracy[0])
print ("precision:-", accuracy[1])
print ("recall:-", accuracy[2])
print ("f1 score:-", accuracy[3])


Epoch 1/3
520/520 [==============================] - 12s 20ms/step - loss: 0.4303
Epoch 2/3
520/520 [==============================] - 10s 20ms/step - loss: 0.2790
Epoch 3/3
520/520 [==============================] - 10s 20ms/step - loss: 0.2169
accuracy:- 0.9350961538461539
precision:- 0.9322195704057279
recall:- 0.9384911100432485
f1 score:- 0.9353448275862069
